In [17]:
import numpy as np
import pandas as pd

In [18]:
df = pd.read_excel('../data/Spline.xlsx', sheet_name = 'Sheet3', dtype={'POSTAL CODE': str})

df['Building'] = df['Building Name'].str.cat(df['POSTAL CODE'], sep = '_')
df = df.drop(columns=['Index', 'Building Name', 'POSTAL CODE', '2020 EUI'])

cols = list(df.columns)
cols.insert(0, cols.pop(cols.index('Building')))

df = df[cols]

df.rename(columns = {'2021 EUI': 'EUI_2021', '2022 EUI': 'EUI_2022', '2023 EUI': 'EUI_2023'}, inplace = True)

In [19]:
df.rename(columns = {'Building Type': 'Type', 'Main Function': 'Function', 'Building Size': 'Size', 'Employee(people)': 'Employee', 'Transportation Data in 2023 (km)': 'Transportation', 'gross floor Area (sqm)': 'GFA'}, inplace = True)

time_series_columns = [
    ('Total Energy Consumption(kwh)_2021', 'Total Energy Consumption(kwh)_2022', 'Total Energy Consumption(kwh)_2023'),
    ('Total Waste Consumed(Metric Tons)_2021', 'Total Waste Consumed(Metric Tons)_2022', 'Total Waste Consumed(Metric Tons)_2023'),
    ('Total Water Consumed (m3) _2021', 'Total Water Consumed (m3) _2022', 'Total Water Consumed (m3) _2023'),
    ('Scope 1 GHG Total Emission(t CO2e)_2021', 'Scope 1 GHG Total Emission(t CO2e)_2022', 'Scope 1 GHG Total Emission(t CO2e)_2023'),
    ('Scope 2 GHG Total Emission(t CO2e)_2021', 'Scope 2 GHG Total Emission(t CO2e)_2022', 'Scope 2 GHG Total Emission(t CO2e)_2023'),
    ('Scope 3 GHG Total Emission(t CO2e)_2021', 'Scope 3 GHG Total Emission(t CO2e)_2022', 'Scope 3 GHG Total Emission(t CO2e)_2023'),
    ('GHG Total Emission(t CO2e)_2021', 'GHG Total Emission(t CO2e)_2022', 'GHG Total Emission(t CO2e)_2023'),
    ('EUI_2021', 'EUI_2022', 'EUI_2023')
]

fixed_columns = ['Building', 'latitude', 'longitude', 'Address', 'Type', 'Function', 'Size', 'Employee', 'Transportation', 'GFA']

df_long = pd.wide_to_long(df, stubnames=['Total Energy Consumption(kwh)', 'Total Waste Consumed(Metric Tons)', 'Total Water Consumed (m3) ', 'Scope 1 GHG Total Emission(t CO2e)', 'Scope 2 GHG Total Emission(t CO2e)', 'Scope 3 GHG Total Emission(t CO2e)', 'GHG Total Emission(t CO2e)', 'EUI'],
                          i=fixed_columns, j='Year', sep='_', suffix='\d+').reset_index()

for col in fixed_columns:
    df_long[col] = df_long[col].fillna(method='ffill')

col_to_move = 'Year'
year_column = df_long.pop(col_to_move)
df_long.insert(1, col_to_move, year_column)


df_long.rename(columns = {'Total Energy Consumption(kwh)': 'Energy', 'Total Waste Consumed(Metric Tons)': 'Waste', 'Total Water Consumed (m3) ': 'Water', 
                     'Scope 1 GHG Total Emission(t CO2e)': 'Scope1', 'Scope 2 GHG Total Emission(t CO2e)': 'Scope2', 'Scope 3 GHG Total Emission(t CO2e)': 'Scope3',
                     'GHG Total Emission(t CO2e)': 'GHG_Total'}, inplace = True)

df_long.head()

<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
C:\Users\86159\AppData\Local\Temp\ipykernel_7128\1356712883.py:17: SyntaxWarning: invalid escape sequence '\d'
  i=fixed_columns, j='Year', sep='_', suffix='\d+').reset_index()
C:\Users\86159\AppData\Local\Temp\ipykernel_7128\1356712883.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_long[col] = df_long[col].fillna(method='ffill')


,Building,Year,latitude,longitude,Address,Type,Function,Size,Employee,Transportation,GFA,Energy,Waste,Water,Scope1,Scope2,Scope3,GHG_Total,EUI
0,GUOCO TOWER_78884,2021,1.2767,103.8452,"1 5 7 9 WALLICH STREET, SINGAPORE 078884",Commercial Building,Office,Large,12000.0,9.187972e+07,129726.0,13286543.0,733.000000,94837.000000,461.685178,6713.963806,6365.373117,13541.022101,226.473043
1,GUOCO TOWER_78884,2022,1.2767,103.8452,"1 5 7 9 WALLICH STREET, SINGAPORE 078884",Commercial Building,Office,Large,12000.0,9.187972e+07,129726.0,13666629.0,917.000000,100404.000000,469.698537,6595.590912,6718.583587,13783.873035,234.805606
2,GUOCO TOWER_78884,2023,1.2767,103.8452,"1 5 7 9 WALLICH STREET, SINGAPORE 078884",Commercial Building,Office,Large,12000.0,9.187972e+07,129726.0,12775562.0,921.000000,113308.000000,109.000000,4527.000000,3945.500000,8581.500000,202.600592
3,SINGAPORE AVIATION ACADEMY_499867,2021,1.3883,103.9903,"1 AVIATION DRIVE, SINGAPORE 499867",Commercial Building,Office,Large,1705.0,1.171629e+07,22368.0,2756143.0,237.290434,64124.820894,64.961373,1281.737448,1030.798679,2377.497500,123.218124
4,SINGAPORE AVIATION ACADEMY_499867,2022,1.3883,103.9903,"1 AVIATION DRIVE, SINGAPORE 499867",Commercial Building,Office,Large,1705.0,1.171629e+07,22368.0,2835047.5,241.473670,64382.446101,65.278484,1314.038774,1046.750446,2426.067704,126.745686


In [20]:
df_long.to_excel('../data/tidy_Spline3.xlsx', index=False)